<a href="https://colab.research.google.com/github/hongik-machine/Default-Risk-Scoring/blob/feat%2Frandom-forest/notebooks/rf_baseline_preprocess_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# ============================================================
# Credit default: 전처리 + RandomForest 파이프라인
# - 파일 업로드 → 클리닝 → train/test 분할 → 파이프라인 전처리 → 학습 → 정확도 출력
# ============================================================

# (필요 시) 패키지 설치
# !pip install -q scikit-learn pandas matplotlib joblib
import pandas as pd
import numpy as np


In [20]:
# ------------------------------------------------------------
# 0) CSV 업로드 (파일 선택)
# ------------------------------------------------------------
from google.colab import files
uploaded = files.upload()
path = list(uploaded.keys())[0]

Saving credit_card_default.csv to credit_card_default (3).csv


In [21]:
# ------------------------------------------------------------
# 1) 데이터 로드 + 기본 점검/클리닝
# ------------------------------------------------------------
def read_csv_kr(p):
    """한국어 CSV 인코딩 흔한 두 가지(utf-8-sig, cp949)를 순차 시도해 안전하게 로드"""
    try:
        return pd.read_csv(p, encoding="utf-8-sig")
    except Exception:
        return pd.read_csv(p, encoding="cp949")

df = read_csv_kr(path)
print("원본 데이터 크기:", df.shape)
# df.head() # 상위 샘플이 필요하면 주석 해제

# 전체 컬럼/결측 상태 확인
print("\n[info]")
print(df.info())
print("\n[결측치 개수]\n", df.isna().sum())

# 1-1) 불필요 인덱스/유사 ID 컬럼 제거
to_drop = [c for c in df.columns if c.lower().startswith("unnamed")]
if to_drop:
    df = df.drop(columns=to_drop)
    print(f"\n드랍한 컬럼: {to_drop}")
print("드랍 후 컬럼 수:", df.shape[1])

# 1-2) 타깃 컬럼 지정 및 타입 정리(0/1 정수)
target = "default_payment_next_month" # 데이터셋의 타깃 이름
assert target in df.columns, "타깃 컬럼이 없습니다. target 변수명을 확인하세요."
df[target] = pd.to_numeric(df[target], errors="coerce").fillna(0).astype(int)

# 1-3) 중복 행 제거
before = len(df)
df = df.drop_duplicates()
print(f"중복 제거: {before - len(df)}행 제거")

# 1-4) 숫자열의 음수값 빠른 점검
num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
neg_counts = (df[num_cols_all] < 0).sum().sort_values(ascending=False)
neg_counts = neg_counts[neg_counts > 0]
if len(neg_counts):
    print("\n[음수값 존재 컬럼 상위]\n", neg_counts.head(10))
else:
    print("\n음수값을 가진 숫자열이 발견되지 않았습니다.")

원본 데이터 크기: (30000, 25)

[info]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  30000 non-null  int64  
 1   limit_bal                   30000 non-null  int64  
 2   sex                         29850 non-null  object 
 3   education                   29850 non-null  object 
 4   marriage                    29850 non-null  object 
 5   age                         29850 non-null  float64
 6   payment_status_sep          30000 non-null  object 
 7   payment_status_aug          30000 non-null  object 
 8   payment_status_jul          30000 non-null  object 
 9   payment_status_jun          30000 non-null  object 
 10  payment_status_may          30000 non-null  object 
 11  payment_status_apr          30000 non-null  object 
 12  bill_statement_sep          30000 non-null  int64  
 13  

In [22]:
# ------------------------------------------------------------
# 2) 학습/검증 분할 (누설 방지: 전처리/스케일링 전에 분할)
# ------------------------------------------------------------
from sklearn.model_selection import train_test_split

X = df.drop(columns=[target]) # 입력 특성
y = df[target] # 타깃(0/1)

# 분류 문제라 클래스 비율을 유지하도록 stratify 사용
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\n학습/검증 크기:", X_train.shape, X_test.shape)
print("학습 타깃 분포:\n", y_train.value_counts(normalize=True).round(3))
print("검증 타깃 분포:\n", y_test.value_counts(normalize=True).round(3))


학습/검증 크기: (23974, 23) (5994, 23)
학습 타깃 분포:
 default_payment_next_month
0    0.779
1    0.221
Name: proportion, dtype: float64
검증 타깃 분포:
 default_payment_next_month
0    0.779
1    0.221
Name: proportion, dtype: float64


In [23]:
# ------------------------------------------------------------
# 3) 전처리 파이프라인 정의
#    - 숫자열: 결측치 평균 대체
#    - 범주열: 결측치 최빈값 대체 + One-Hot 인코딩
#    - RandomForest는 트리 기반이라 스케일링이 필수는 아님
# ------------------------------------------------------------
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# 숫자/범주형 컬럼 자동 식별
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(exclude=[np.number]).columns.tolist()
print("\n숫자형:", len(num_cols), "| 범주형:", len(cat_cols))
if cat_cols:
    print("범주형 예시:", cat_cols[:5])

numeric_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")) # 숫자 결측 → 평균으로 대체
])
categorical_processor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), # 범주 결측 → 최빈값
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)) # 미지의 범주 무시, 밀집행렬 반환
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_processor, num_cols),  # 숫자 파이프라인
        ("cat", categorical_processor, cat_cols) # 범주 파이프라인
    ],
    remainder="drop"  # 지정 안 된 컬럼이 있으면 드롭(안전)
)



숫자형: 14 | 범주형: 9
범주형 예시: ['sex', 'education', 'marriage', 'payment_status_sep', 'payment_status_aug']


In [24]:
# ------------------------------------------------------------
# 4) 모델: RandomForestClassifier
#    - class_weight="balanced"로 소수 클래스 가중 (불균형 완화)
#    - max_depth, min_samples_leaf 조절로 과적합 제어 가능(필요 시)
# ------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=400, # 트리 개수
    max_depth=None, # 과적합 보이면 10~20 등으로 제한
    min_samples_split=2, # 분할 최소 샘플 수
    min_samples_leaf=1, # 리프 최소 샘플 수
    max_features="sqrt",  # 각 분할 시 고려할 특성 수
    class_weight="balanced",  # 클래스 불균형 대비
    random_state=42,  # 재현성 고정
    n_jobs=-1  # 멀티코어 활용
)

# 파이프라인: 전처리 → 모델 (누설 방지, 재사용/서빙에도 용이)
pipe = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", clf)
])


In [27]:
# ------------------------------------------------------------
# 5) 학습 및 간단 정확도(Training/Test) 출력
# ------------------------------------------------------------
pipe.fit(X_train, y_train) # 전처리+모델을 한 번에 학습
print("\n학습 완료") # 학습 끝 신호

# 분류 문제에서 Pipeline.score = 정확도(accuracy)
train_acc = pipe.score(X_train, y_train) # 훈련 정확도
test_acc  = pipe.score(X_test, y_test) # 테스트 정확도
print(f"Training Accuracy: {train_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


학습 완료
Training Accuracy: 0.9995
Test Accuracy: 0.8123
